In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
random.seed(12)

# Task 1: Build and load the dataset
Create a list of dictionaries called attendance_raw with exactly 24 records. Each record must include:

student_id in the format S001 to S024
cohort as one of ["alpha", "beta", "gamma"]
attended_sessions as an integer between 0 and 6
expected_sessions as the integer 6
Then load the list into a DataFrame named attendance. Print the first five rows and call info() to confirm the structure and data types.

In [3]:
cohorts = ['alpha', 'beta', 'gamma']
attendance_raw = []

for i in range(1, 25):
    record = {
        'student_id' : f"S{i:03d}",
        'cohort' : cohorts[(i - 1) % 3],
        'attended_sessions' : random.randint(0, 6),
        'expected_sessions' : 6
    }
    attendance_raw.append(record)

In [4]:
attendance = pd.DataFrame(attendance_raw)

In [5]:
attendance.head()

,student_id,cohort,attended_sessions,expected_sessions
0,S001,alpha,3,6
1,S002,beta,2,6
2,S003,gamma,5,6
3,S004,alpha,4,6
4,S005,beta,5,6


In [6]:
attendance.info()

<class 'pandas.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   student_id         24 non-null     str  
 1   cohort             24 non-null     str  
 2   attended_sessions  24 non-null     int64
 3   expected_sessions  24 non-null     int64
dtypes: int64(2), str(2)
memory usage: 900.0 bytes


# Task 2: Set an index and validate alignment
Set student_id as the index and store the result in attendance_indexed. Create a Series named excused_absences with at least 10 student IDs (some IDs must not exist in the DataFrame). Add this Series to attended_sessions to create a new column adjusted_attendance. Confirm that rows without matching IDs become missing in adjusted_attendance. Then fill missing values in adjusted_attendance with the original attended_sessions and show the updated column.

In [7]:
attendance_indexed = attendance.set_index('student_id')
attendance_indexed

,cohort,attended_sessions,expected_sessions
student_id,,,
S001,alpha,3,6
S002,beta,2,6
S003,gamma,5,6
S004,alpha,4,6
S005,beta,5,6
S006,gamma,2,6
S007,alpha,1,6
S008,beta,3,6
S009,gamma,0,6


In [8]:
excused_data = {
    'S001' : 1, 'S003' : 2, "S005": 1, "S008": 1, "S010": 2,
    "S012": 1, "S015": 1, "S025": 1, "S030": 2, "S099": 1
}
excused_absences = pd.Series(excused_data, name='excused')
excused_absences

S001    1
S003    2
S005    1
S008    1
S010    2
S012    1
S015    1
S025    1
S030    2
S099    1
Name: excused, dtype: int64

In [9]:
attendance_indexed['adjusted_attendance'] = attendance_indexed['attended_sessions'] + excused_absences
attendance_indexed

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S001,alpha,3,6,4.0
S002,beta,2,6,NaN
S003,gamma,5,6,7.0
S004,alpha,4,6,NaN
S005,beta,5,6,6.0
S006,gamma,2,6,NaN
S007,alpha,1,6,NaN
S008,beta,3,6,4.0
S009,gamma,0,6,NaN


In [10]:
attendance_indexed[attendance_indexed["adjusted_attendance"].isna()]

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S002,beta,2,6,NaN
S004,alpha,4,6,NaN
S006,gamma,2,6,NaN
S007,alpha,1,6,NaN
S009,gamma,0,6,NaN
S011,beta,3,6,NaN
S013,alpha,5,6,NaN
S014,beta,6,6,NaN
S016,alpha,5,6,NaN


In [11]:
attendance_indexed['adjusted_attendance'] = attendance_indexed['adjusted_attendance'].fillna(attendance_indexed["attended_sessions"])

In [12]:
attendance_indexed['adjusted_attendance'] = attendance_indexed['adjusted_attendance'].astype('int64')

In [13]:
attendance_indexed

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S001,alpha,3,6,4
S002,beta,2,6,2
S003,gamma,5,6,7
S004,alpha,4,6,4
S005,beta,5,6,6
S006,gamma,2,6,2
S007,alpha,1,6,1
S008,beta,3,6,4
S009,gamma,0,6,0


# Task 3: Clean and normalize categories
Introduce a small inconsistency by modifying a few cohort values to include extra whitespace and inconsistent casing. Then write pandas code to normalize the cohort column by stripping whitespace and converting to lowercase. After cleaning, display the unique cohorts to confirm that the inconsistencies are resolved.

In [14]:
attendance_indexed.iloc[0, 0] = ' Alpha'
attendance_indexed.iloc[1, 0] = 'BETA '
attendance_indexed.iloc[2, 0] = 'gaMMa'

In [15]:
attendance_indexed['cohort'] = attendance_indexed['cohort'].str.strip().str.lower()

In [16]:
attendance_indexed['cohort'].unique()

<StringArray>
['alpha', 'beta', 'gamma']
Length: 3, dtype: str

# Task 4: Filter and compute summaries
Filter the DataFrame to students where attended_sessions is below expected_sessions. Store the result in low_attendance. Compute the average attended_sessions by cohort using groupby. Print the summary and verify that cohorts in the summary match the cleaned cohorts.

In [17]:
low_attendance = attendance_indexed[attendance_indexed['attended_sessions'] < attendance_indexed['expected_sessions']]
low_attendance

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S001,alpha,3,6,4
S002,beta,2,6,2
S003,gamma,5,6,7
S004,alpha,4,6,4
S005,beta,5,6,6
S006,gamma,2,6,2
S007,alpha,1,6,1
S008,beta,3,6,4
S009,gamma,0,6,0


In [18]:
cohort_summary = attendance_indexed.groupby('cohort')['attended_sessions'].mean()

print(cohort_summary)

cohort
alpha    3.250
beta     4.125
gamma    2.125
Name: attended_sessions, dtype: float64


In [19]:
print(f"Summary cohorts: {list(cohort_summary.index)}")
print(f"Expected cohorts: {list(attendance_indexed['cohort'].unique())}")

Summary cohorts: ['alpha', 'beta', 'gamma']
Expected cohorts: ['alpha', 'beta', 'gamma']


# Task 5: Add a derived field and validate it
Create a new column attendance_ok that is True when attended_sessions is at least expected_sessions, otherwise False. Use a boolean comparison rather than a loop. Then validate the column by confirming that every row in low_attendance has attendance_ok equal to False.

In [20]:
attendance_indexed['attendance_ok'] = attendance_indexed['attended_sessions'] >= attendance_indexed['expected_sessions']

In [21]:
low_attendance = low_attendance.copy()
low_attendance['attendance_ok'] = attendance_indexed['attendance_ok']
low_attendance

,cohort,attended_sessions,expected_sessions,adjusted_attendance,attendance_ok
student_id,,,,,
S001,alpha,3,6,4,False
S002,beta,2,6,2,False
S003,gamma,5,6,7,False
S004,alpha,4,6,4,False
S005,beta,5,6,6,False
S006,gamma,2,6,2,False
S007,alpha,1,6,1,False
S008,beta,3,6,4,False
S009,gamma,0,6,0,False


In [22]:
is_valid = (low_attendance['attendance_ok'] == False).all()
print(f"Validation: {is_valid}")

Validation: True
